In [96]:
from call_functions import *
import matplotlib.pyplot as plt
import plotly.express as px

In [121]:
# ------------------------------
# Parameters to tune
# ------------------------------

x0 = 0
L = 2 * np.pi
Nx = 2**10 +1
Nf = Nx // 2
dt = 1e-3
a = 1
g = 1
gain = 1
T_start = 0
T = 500
N_ini_fq = 2**2

# ------------------------------
# Space discretization
# ------------------------------

dx = L / Nx
Lx = L - dx
xspan = np.arange(x0, Lx + dx, dx)
frequences = np.fft.fftfreq(Nx, dx) * 2 * np.pi
frequences2 = fftfreq(Nx, dx) * 2 * np.pi # same as above

# ------------------------------
# Time discretization
# ------------------------------

T_end = T_start + T
tspan = np.arange(T_start, T_end + dt, dt)
Nt = len(tspan)

In [122]:
frequences

array([ 0.,  1.,  2., ..., -3., -2., -1.])

In [127]:
# 1) DFT and iDFT Matrices (as well as no mean matrices)
DFT = 1 / Nx * getDFT(Nx)
iDFT = getiDFT(Nx)

DFT_n_m = 1 / Nx * getDFT_no_mean(Nx)
iDFT_n_m = getiDFT_no_mean(Nx)

# 2) True matrix A not in Fourier
O = np.zeros((Nx, Nx))
I = np.eye(Nx)
G = Matrix_A_dz(Nx, dx)
_, C = GetC(np.abs(xspan - np.pi) <= a)
obs_space = np.abs(xspan - np.pi) <= a
C = -np.dot(C.T, C)
A = np.block([[O, -I], [G, C]])

# True matrix A in Fourier
fO = np.zeros((2 * Nf + 1, 2 * Nf + 1))
fI = -np.eye(2 * Nf + 1)
fF = np.diag(np.abs(np.concatenate((np.arange(0, Nf + 1), np.arange(-Nf, 0)))) * 2 * np.pi)
frequences = fftfreq(Nx, dx) * 2 * np.pi  # Assuming the previously defined fftfreq function
fC = np.real(-((a * np.exp((-1j * np.pi) * (frequences.reshape(-1, 1) - frequences))) / np.pi) * np.sinc(a * (frequences.reshape(-1, 1) - frequences) / np.pi))

In [124]:
# Now : DFT @ fF @ iDFT = G * np.pi * 2

np.linalg.norm(np.real(iDFT @ fF @ DFT) - G * np.pi * 2)/np.linalg.norm(G * np.pi * 2) < 1e-12

False

In [125]:
# Now : DFT @ fF @ iDFT = G * np.pi * 2

np.linalg.norm(np.real(iDFT @ C @ DFT) - fC)/np.linalg.norm(fC)

0.04742889729166587

In [126]:
px.scatter(x=frequences,y=[np.real(iDFT @ C @ DFT)[0,:],fC[0,:]])

In [44]:
fA = np.block([[fO, fI], [fF, fC]])

In [46]:
fA

array([[ 0.        ,  0.        ,  0.        , -1.        , -0.        ,
        -0.        ],
       [ 0.        ,  0.        ,  0.        , -0.        , -1.        ,
        -0.        ],
       [ 0.        ,  0.        ,  0.        , -0.        , -0.        ,
        -1.        ],
       [ 0.        ,  0.        ,  0.        , -0.31830989,  0.26784853,
         0.26784853],
       [ 0.        ,  6.28318531,  0.        ,  0.26784853, -0.31830989,
        -0.14471918],
       [ 0.        ,  0.        ,  6.28318531,  0.26784853, -0.14471918,
        -0.31830989]])